## Data extraction: daily, plankton

**References**:
- https://machinelearningmastery.com/sarima-for-time-series-forecasting-in-python/
- https://otexts.com/fpp2/stationarity.html

Some notes....
  - Seasonal Autoregressive Integrated Moving Average (SARIMA): method for time series forecasting with univariate data containing trends and seasonality.
  - I may want to simply compute the Granger Causality for my time series.

#### Import modules

In [96]:
import netCDF4
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from pathlib import Path
import os
import seaborn as sns
import matplotlib.pyplot as plt
import xarray as xr
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
import plotly.graph_objects as go

#### Define lists called in the forthcoming for loops

In [2]:
sector = ['NB', 'CB', 'SB']
time_code = ['00:00','03:00','06:00','09:00','12:00','15:00','18:00','21:00']
date_2013 = ['January_4750','February_4780','March_4810','April_4840','May_4870','June_4900',
             'July_4930','August_4960','September_4990','October_5020','November_5050', 'December_5080']
date_2014 = ['January_5115','February_5145','March_5175','April_5205','May_5235','June_5265',
             'July_5295','August_5325','September_5355','October_5385','November_5415', 'December_5445']
day_2013 = ['4750','4780','4810','4840','4870','4900','4930','4960','4990','5020','5050', '5080']
day_2014 = ['5115','5145','5175','5205','5235','5265','5295','5325','5355','5385','5415', '5445']

## Loop through all months for each sector: 2013

In [38]:
# Reset the working directory before running the loops
%pwd
%cd /home/lindsay/hioekg-2013/

# Add master list for the files collected by this first nested loop and group lists
file_list = []
nsmz=[]
nmdz=[]
nlgz=[]
nsm=[]
nlg=[]

for i in range(0,12):
    for j in range(0,3):
        folder = '/home/lindsay/hioekg-2013/' + str(date_2013[i]) + '/'
        os.chdir(folder)
        file = xr.open_dataset(str(sector[j]) + '_all_var_surface_0' + str(day_2013[i]) + '.nc')
        file_list.append(file)      

for item in file_list: 
# nsmz: Small zooplankton
    dat = item.nsmz
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nsmz.append(dat_numpy)
# nmdz: Medium zooplankton
    dat = item.nmdz
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nmdz.append(dat_numpy)
# nlgz: Large zooplankton
    dat = item.nlgz
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nlgz.append(dat_numpy)
# nsm: Small phytoplankton
    dat = item.nsm
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nsm.append(dat_numpy)
# nlg: Large phytoplankton
    dat = item.nlg
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nlg.append(dat_numpy)

/home/lindsay/hioekg-2013


Inspecting the elements of file_list indicates that the order of the arrays in my variable lists are as such:

- Jan_NB, Jan_CB, Jan_SB, Feb_NB, Feb_CB, Feb_SB...and so on.

My path forward:
- Loop through each array element in each list
- For elements 0-2, must join array with a Date list of '01-01-2013', 3-5 '02-01-2013'...
- All elements joined with alternating Sector lists of NB, CB, SB...

My approach at this point is to extract elements in groups of 3 and tailor each for loop accordingly.

Then, run this loop for 2014, and for the other variables, and append those lists to this master df. This will make plotting and further manipulation pretty easy.

#### Classifying values by time, sector

In [39]:
# Creating 12-unit date list and month list, 3-unit sector list, and empty list to store organized values:
date = ['01-01-2013','02-01-2013','03-01-2013','04-01-2013','05-01-2013','06-01-2013','07-01-2013','08-01-2013',
        '09-01-2013','10-01-2013','11-01-2013','12-01-2013']
#month=['01','02','03','04','05','06','07','08','09','10','11','12']
month=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
sector=['NB','CB','SB']

# Store my dfs separately for my own benefit in error reduction
list_for_dataframe_nsmz_2013=[]
list_for_dataframe_nmdz_2013=[]
list_for_dataframe_nlgz_2013=[]
list_for_dataframe_nsm_2013=[]
list_for_dataframe_nlg_2013=[]

#### nsmz (2013)

In [40]:
%cd /home/lindsay/hioekg-2013

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nsmz):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nsmz_2013.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nsmz_df_2013 = pd.DataFrame(list_for_dataframe_nsmz_2013)
# Add group identifier and year column
nsmz_df_2013['group']='nsmz'
nsmz_df_2013['year']=2013
print(len(nsmz_df_2013))
nsmz_df_2013.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nsmz_df_2013,'nsmz_df_2013.csv')

/home/lindsay/hioekg-2013
77808
/home/lindsay/hioekg-compare-years


#### nmdz (2013)

In [41]:
%cd /home/lindsay/hioekg-2013

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nmdz):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nmdz_2013.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nmdz_df_2013 = pd.DataFrame(list_for_dataframe_nmdz_2013)
# Add group identifier and year column
nmdz_df_2013['group']='nmdz'
nmdz_df_2013['year']=2013
print(len(nmdz_df_2013))
nmdz_df_2013.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nmdz_df_2013,'nmdz_df_2013.csv')

/home/lindsay/hioekg-2013
77808
/home/lindsay/hioekg-compare-years


#### nlgz (2013)

In [42]:
%cd /home/lindsay/hioekg-2013

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nlgz):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nlgz_2013.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nlgz_df_2013 = pd.DataFrame(list_for_dataframe_nlgz_2013)
# Add group identifier and year column
nlgz_df_2013['group']='nlgz'
nlgz_df_2013['year']=2013
print(len(nlgz_df_2013))
nlgz_df_2013.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nlgz_df_2013,'nlgz_df_2013.csv')

/home/lindsay/hioekg-2013
77808
/home/lindsay/hioekg-compare-years


#### nsm

In [43]:
%cd /home/lindsay/hioekg-2013

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nsm):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nsm_2013.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nsm_df_2013 = pd.DataFrame(list_for_dataframe_nsm_2013)
# Add group identifier and year column
nsm_df_2013['group']='nsm'
nsm_df_2013['year']=2013
print(len(nsm_df_2013))
nsm_df_2013.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nsm_df_2013,'nsm_df_2013.csv')

/home/lindsay/hioekg-2013
77808
/home/lindsay/hioekg-compare-years


#### nlg

In [44]:
%cd /home/lindsay/hioekg-2013

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nlg):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nlg_2013.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nlg_df_2013 = pd.DataFrame(list_for_dataframe_nlg_2013)
# Add group identifier and year column
nlg_df_2013['group']='nlg'
nlg_df_2013['year']=2013
print(len(nlg_df_2013))
nlg_df_2013.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nlg_df_2013,'nlg_df_2013.csv')

/home/lindsay/hioekg-2013
77808
/home/lindsay/hioekg-compare-years


#### Concatenate all dataframes

In [45]:
all_frames = [nsmz_df_2013,nmdz_df_2013,nlgz_df_2013,nsm_df_2013,nlg_df_2013]

df_2013 = pd.concat(all_frames, axis=0)
df_2013.tail()

,month,date,sector,concentration,group,year
77803,Dec,12-01-2013,SB,7.168995e-08,nlg,2013
77804,Dec,12-01-2013,SB,7.165477e-08,nlg,2013
77805,Dec,12-01-2013,SB,7.163389e-08,nlg,2013
77806,Dec,12-01-2013,SB,7.162012e-08,nlg,2013
77807,Dec,12-01-2013,SB,7.161326e-08,nlg,2013


## Loop through each sector: 2014

In [46]:
# Reset the working directory before running the loops
%pwd
%cd /home/lindsay/hioekg-2014/

# Add master list for the files collected by this first nested loop and group lists
file_list = []
nsmz=[]
nmdz=[]
nlgz=[]
nsm=[]
nlg=[]

for i in range(0,12):
    for j in range(0,3):
        folder = '/home/lindsay/hioekg-2014/' + str(date_2014[i]) + '/'
        os.chdir(folder)
        file = xr.open_dataset(str(sector[j]) + '_all_var_surface_0' + str(day_2014[i]) + '.nc')
        file_list.append(file)      

for item in file_list: 
# nsmz: Small zooplankton
    dat = item.nsmz
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nsmz.append(dat_numpy)
# nmdz: Medium zooplankton
    dat = item.nmdz
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nmdz.append(dat_numpy)
# nlgz: Large zooplankton
    dat = item.nlgz
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nlgz.append(dat_numpy)
# nsm: Small phytoplankton
    dat = item.nsm
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nsm.append(dat_numpy)
# nlg: Large phytoplankton
    dat = item.nlg
    dat_numpy = dat.values
    dat_numpy=dat_numpy[~np.isnan(dat_numpy)]
    nlg.append(dat_numpy)

/home/lindsay/hioekg-2014


In [47]:
# Creating 12-unit date list and month list, 3-unit sector list, and empty list to store organized values:
date = ['01-01-2014','02-01-2014','03-01-2014','04-01-2014','05-01-2014','06-01-2014','07-01-2014','08-01-2014',
        '09-01-2014','10-01-2014','11-01-2014','12-01-2014']
#month=['01','02','03','04','05','06','07','08','09','10','11','12']
month=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
sector=['NB','CB','SB']

# Store my dfs separately for my own benefit in error reduction
list_for_dataframe_nsmz_2014=[]
list_for_dataframe_nmdz_2014=[]
list_for_dataframe_nlgz_2014=[]
list_for_dataframe_nsm_2014=[]
list_for_dataframe_nlg_2014=[]

#### nsmz (2014)

In [48]:
%cd /home/lindsay/hioekg-2014

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nsmz):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nsmz_2014.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nsmz_df_2014 = pd.DataFrame(list_for_dataframe_nsmz_2014)
# Add group identifier and year column
nsmz_df_2014['group']='nsmz'
nsmz_df_2014['year']=2014
print(len(nsmz_df_2014))
nsmz_df_2014.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nsmz_df_2014,'nsmz_df_2014.csv')

/home/lindsay/hioekg-2014
77808
/home/lindsay/hioekg-compare-years


#### nmdz (2014)

In [49]:
%cd /home/lindsay/hioekg-2014

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nmdz):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nmdz_2014.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nmdz_df_2014 = pd.DataFrame(list_for_dataframe_nmdz_2014)
# Add group identifier and year column
nmdz_df_2014['group']='nmdz'
nmdz_df_2014['year']=2014
print(len(nmdz_df_2014))
nmdz_df_2014.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nmdz_df_2014,'nmdz_df_2014.csv')

/home/lindsay/hioekg-2014
77808
/home/lindsay/hioekg-compare-years


#### nlgz (2014)

In [50]:
%cd /home/lindsay/hioekg-2014

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nlgz):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nlgz_2014.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nlgz_df_2014 = pd.DataFrame(list_for_dataframe_nlgz_2014)
# Add group identifier and year column
nlgz_df_2014['group']='nlgz'
nlgz_df_2014['year']=2014
print(len(nlgz_df_2014))
nlgz_df_2014.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nlgz_df_2014,'nlgz_df_2014.csv')

/home/lindsay/hioekg-2014
77808
/home/lindsay/hioekg-compare-years


#### nsm (2014)

In [51]:
%cd /home/lindsay/hioekg-2014

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nsm):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nsm_2014.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nsm_df_2014 = pd.DataFrame(list_for_dataframe_nsm_2014)
# Add group identifier and year column
nsm_df_2014['group']='nsm'
nsm_df_2014['year']=2014
print(len(nsm_df_2014))
nsm_df_2014.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nsm_df_2014,'nsm_df_2014.csv')

/home/lindsay/hioekg-2014
77808
/home/lindsay/hioekg-compare-years


#### nlg (2014)

In [52]:
%cd /home/lindsay/hioekg-2014

# Iterating sequentially (i) through each array (element) in the list of arrays (selection)
for i,element in enumerate(nlg):
    this_date = date[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_month = month[i//3] # integer division: 0,1,2//3 returns same val (0); 3,4,5 returns same val (1)...
    this_sector = sector[i%3] # this moves faster- every other element, sequence of 3
    for sub_element in element:
        list_for_dataframe_nlg_2014.append(
            {'month': this_month, 'date': this_date, 'sector': this_sector, 'concentration': sub_element})

nlg_df_2014 = pd.DataFrame(list_for_dataframe_nlg_2014)
# Add group identifier and year column
nlg_df_2014['group']='nlg'
nlg_df_2014['year']=2014
print(len(nlg_df_2014))
nlg_df_2014.head()

%cd /home/lindsay/hioekg-compare-years/
pd.DataFrame.to_csv(nlg_df_2014,'nlg_df_2014.csv')

/home/lindsay/hioekg-2014
77808
/home/lindsay/hioekg-compare-years


#### Concatenate all 2014 dataframes

In [53]:
all_frames = [nsmz_df_2014,nmdz_df_2014,nlgz_df_2014,nsm_df_2014,nlg_df_2014]

df_2014 = pd.concat(all_frames, axis=0)

In [54]:
df_2014.head()
pd.DataFrame.to_csv(df_2014,'df_2014.csv')

#### Concatenate the large year-distinct dfs

In [55]:
yrs = [df_2013,df_2014]
df_all = pd.concat(yrs, axis=0)

#### Write dfs to file

In [56]:
%cd /home/lindsay/hioekg-compare-years/

pd.DataFrame.to_csv(df_all, 'df_all.csv')

/home/lindsay/hioekg-compare-years


##### For use if/when the kernel dies:

In [57]:
%cd /home/lindsay/hioekg-compare-years/

nsmz_df_2013=pd.read_csv('nsmz_df_2013.csv')
nmdz_df_2013=pd.read_csv('nmdz_df_2013.csv')
nlgz_df_2013=pd.read_csv('nlgz_df_2013.csv')
nsm_df_2013=pd.read_csv('nsm_df_2013.csv')
nlg_df_2013=pd.read_csv('nlg_df_2013.csv')

nsmz_df_2014=pd.read_csv('nsmz_df_2014.csv')
nmdz_df_2014=pd.read_csv('nmdz_df_2014.csv')
nlgz_df_2014=pd.read_csv('nlgz_df_2014.csv')
nsm_df_2014=pd.read_csv('nsm_df_2014.csv')
nlg_df_2014=pd.read_csv('nlg_df_2014.csv')

df_all=pd.read_csv('df_all.csv')

/home/lindsay/hioekg-compare-years


### Statistical fun(ctions)

I really care about the large zooplankton. So let's see if I should care, statistically speaking.

#### Paired sample t-test *or* Wilcoxon signed-rank test
I want to see if there are differences in the mean concentrations of each group comparing the same month in 2013 v. 2014.
- The t score is a ratio between the difference between two groups and the difference within the groups. The larger the t score, the more difference there is between groups. The smaller the t score, the more similarity there is between groups. A t score of 3 means that the groups are three times as different from each other as they are within each other. When you run a t test, the bigger the t-value, the more likely it is that the results are repeatable.
- I am going to look at the month-to-month t-scores for each group.
- The paired t-test assumes normality of the differences between groups. To test this, I am going run the Shapiro-Wilk test for normality. If the [W-value is small](https://emilkirkegaard.dk/en/?p=4452) (<0.99) and the p-value is significant (<0.05), the group fails the test and the Wilcoxon signed-rank test is a good non-parametric alternative.

In [76]:
nsmz_yrs = [nsmz_df_2013,nsmz_df_2014]
nmdz_yrs = [nmdz_df_2013,nmdz_df_2014]
nlgz_yrs = [nlgz_df_2013,nlgz_df_2014]
nsm_yrs = [nsm_df_2013,nsm_df_2014]
nlg_yrs = [nlg_df_2013,nlg_df_2014]

nsmz_df = pd.concat(nsmz_yrs, axis=0)
nmdz_df = pd.concat(nmdz_yrs, axis=0)
nlgz_df = pd.concat(nlgz_yrs, axis=0)
nsm_df = pd.concat(nsm_yrs, axis=0)
nlg_df = pd.concat(nlg_yrs, axis=0)

/home/lindsay/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys
/home/lindsay/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/home/lindsay/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the w

#### Testing for normality for each group year-to-year

In [79]:
nsmz_df_2014['nsmz_difference'] = abs(nsmz_df_2014['concentration'] - nsmz_df_2013['concentration'])
print(str('nsmz:'))
print(stats.shapiro(nsmz_df_2014['nsmz_difference']))

nmdz_df_2014['nmdz_difference'] = abs(nmdz_df_2014['concentration'] - nmdz_df_2013['concentration'])
print(str('nmdz:'))
print(stats.shapiro(nmdz_df_2014['nmdz_difference']))

nlgz_df_2014['nlgz_difference'] = abs(nlgz_df_2014['concentration'] - nlgz_df_2013['concentration'])
print(str('nlgz:'))
print(stats.shapiro(nlgz_df_2014['nlgz_difference']))

nsm_df_2014['nsm_difference'] = abs(nsm_df_2014['concentration'] - nsm_df_2013['concentration'])
print(str('nsm:'))
print(stats.shapiro(nsm_df_2014['nsm_difference']))

nlg_df_2014['nlg_difference'] = abs(nlg_df_2014['concentration'] - nlg_df_2013['concentration'])
print(str('nlg:'))
print(stats.shapiro(nlg_df_2014['nlg_difference']))

nsmz:
(0.19189798831939697, 0.0)
nmdz:
(0.6648262143135071, 0.0)
nlgz:
(0.8492623567581177, 0.0)
nsm:
(0.7858284711837769, 0.0)
nlg:
(0.44932371377944946, 0.0)


Indeed, I can't assume normality from any of these groups, and that is fine and not terribly surprising. 

#### Wilcoxon sign-ranked test

Assumptions are all met:
- The dependent variable (DV) must be continuous which is measured on an ordinal or continuous scale
- The paired observations are randomly and independently drawn
- The paired observations come from the same population

I am interested in how statistically important the differences in monthly means are for the same group in different years.

##### nsmz

In [80]:
nsmz_Jan_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Jan']
nsmz_Feb_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Feb']
nsmz_Mar_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Mar']
nsmz_Apr_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Apr']
nsmz_May_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='May']
nsmz_Jun_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Jun']
nsmz_Jul_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Jul']
nsmz_Aug_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Aug']
nsmz_Sep_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Sep']
nsmz_Oct_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Oct']
nsmz_Nov_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Nov']
nsmz_Dec_2013= nsmz_df_2013.loc[nsmz_df_2013['month']=='Dec']

nsmz_Jan_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Jan']
nsmz_Feb_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Feb']
nsmz_Mar_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Mar']
nsmz_Apr_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Apr']
nsmz_May_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='May']
nsmz_Jun_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Jun']
nsmz_Jul_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Jul']
nsmz_Aug_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Aug']
nsmz_Sep_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Sep']
nsmz_Oct_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Oct']
nsmz_Nov_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Nov']
nsmz_Dec_2014= nsmz_df_2014.loc[nsmz_df_2014['month']=='Dec']

In [81]:
print(str('Jan nsmz:'))
print(stats.wilcoxon(nsmz_Jan_2014['nsmz_difference']))
print(str('Feb nsmz:'))
print(stats.wilcoxon(nsmz_Feb_2014['nsmz_difference']))    
print(str('Mar nsmz:'))
print(stats.wilcoxon(nsmz_Mar_2014['nsmz_difference']))  
print(str('Apr nsmz:'))
print(stats.wilcoxon(nsmz_Apr_2014['nsmz_difference']))
print(str('May nsmz:'))
print(stats.wilcoxon(nsmz_May_2014['nsmz_difference']))  
print(str('Jun nsmz:'))
print(stats.wilcoxon(nsmz_Jun_2014['nsmz_difference']))   
print(str('Jul nsmz:'))
print(stats.wilcoxon(nsmz_Jul_2014['nsmz_difference']))
print(str('Aug nsmz:'))
print(stats.wilcoxon(nsmz_Aug_2014['nsmz_difference']))   
print(str('Sep nsmz:'))
print(stats.wilcoxon(nsmz_Sep_2014['nsmz_difference']))    
print(str('Oct nsmz:'))
print(stats.wilcoxon(nsmz_Oct_2014['nsmz_difference']))
print(str('Nov nsmz:'))
print(stats.wilcoxon(nsmz_Nov_2014['nsmz_difference']))   
print(str('Dec nsmz:'))
print(stats.wilcoxon(nsmz_Dec_2014['nsmz_difference']))

Jan nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Feb nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Mar nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Apr nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
May nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Jun nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Jul nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Aug nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Sep nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Oct nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Nov nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Dec nsmz:
WilcoxonResult(statistic=0.0, pvalue=0.0)


##### nmdz

In [82]:
nmdz_Jan_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Jan']
nmdz_Feb_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Feb']
nmdz_Mar_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Mar']
nmdz_Apr_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Apr']
nmdz_May_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='May']
nmdz_Jun_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Jun']
nmdz_Jul_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Jul']
nmdz_Aug_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Aug']
nmdz_Sep_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Sep']
nmdz_Oct_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Oct']
nmdz_Nov_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Nov']
nmdz_Dec_2013= nmdz_df_2013.loc[nmdz_df_2013['month']=='Dec']

nmdz_Jan_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Jan']
nmdz_Feb_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Feb']
nmdz_Mar_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Mar']
nmdz_Apr_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Apr']
nmdz_May_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='May']
nmdz_Jun_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Jun']
nmdz_Jul_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Jul']
nmdz_Aug_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Aug']
nmdz_Sep_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Sep']
nmdz_Oct_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Oct']
nmdz_Nov_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Nov']
nmdz_Dec_2014= nmdz_df_2014.loc[nmdz_df_2014['month']=='Dec']

print(str('Jan nmdz:'))
print(stats.wilcoxon(nmdz_Jan_2014['nmdz_difference']))
print(str('Feb nmdz:'))
print(stats.wilcoxon(nmdz_Feb_2014['nmdz_difference']))    
print(str('Mar nmdz:'))
print(stats.wilcoxon(nmdz_Mar_2014['nmdz_difference']))  
print(str('Apr nmdz:'))
print(stats.wilcoxon(nmdz_Apr_2014['nmdz_difference']))
print(str('May nmdz:'))
print(stats.wilcoxon(nmdz_May_2014['nmdz_difference']))  
print(str('Jun nmdz:'))
print(stats.wilcoxon(nmdz_Jun_2014['nmdz_difference']))   
print(str('Jul nmdz:'))
print(stats.wilcoxon(nmdz_Jul_2014['nmdz_difference']))
print(str('Aug nmdz:'))
print(stats.wilcoxon(nmdz_Aug_2014['nmdz_difference']))   
print(str('Sep nmdz:'))
print(stats.wilcoxon(nmdz_Sep_2014['nmdz_difference']))    
print(str('Oct nmdz:'))
print(stats.wilcoxon(nmdz_Oct_2014['nmdz_difference']))
print(str('Nov nmdz:'))
print(stats.wilcoxon(nmdz_Nov_2014['nmdz_difference']))   
print(str('Dec nmdz:'))
print(stats.wilcoxon(nmdz_Dec_2014['nmdz_difference']))

Jan nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Feb nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Mar nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Apr nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
May nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Jun nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Jul nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Aug nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Sep nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Oct nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Nov nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Dec nmdz:
WilcoxonResult(statistic=0.0, pvalue=0.0)


#### nlgz

In [87]:
nlgz_Jan_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Jan']
nlgz_Feb_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Feb']
nlgz_Mar_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Mar']
nlgz_Apr_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Apr']
nlgz_May_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='May']
nlgz_Jun_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Jun']
nlgz_Jul_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Jul']
nlgz_Aug_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Aug']
nlgz_Sep_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Sep']
nlgz_Oct_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Oct']
nlgz_Nov_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Nov']
nlgz_Dec_2013= nlgz_df_2013.loc[nlgz_df_2013['month']=='Dec']

nlgz_Jan_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Jan']
nlgz_Feb_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Feb']
nlgz_Mar_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Mar']
nlgz_Apr_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Apr']
nlgz_May_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='May']
nlgz_Jun_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Jun']
nlgz_Jul_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Jul']
nlgz_Aug_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Aug']
nlgz_Sep_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Sep']
nlgz_Oct_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Oct']
nlgz_Nov_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Nov']
nlgz_Dec_2014= nlgz_df_2014.loc[nlgz_df_2014['month']=='Dec']

print(str('Jan nlgz:'))
print(stats.wilcoxon(nlgz_Jan_2014['nlgz_difference'], zero_method='pratt'))
print(str('Feb nlgz:'))
print(stats.wilcoxon(nlgz_Feb_2014['nlgz_difference'], zero_method='pratt'))    
print(str('Mar nlgz:'))
print(stats.wilcoxon(nlgz_Mar_2014['nlgz_difference'], zero_method='pratt'))  
print(str('Apr nlgz:'))
print(stats.wilcoxon(nlgz_Apr_2014['nlgz_difference'], zero_method='pratt'))
print(str('May nlgz:'))
print(stats.wilcoxon(nlgz_May_2014['nlgz_difference'], zero_method='pratt'))  
print(str('Jun nlgz:'))
print(stats.wilcoxon(nlgz_Jun_2014['nlgz_difference'], zero_method='pratt'))  
print(str('Jul nlgz:'))
print(stats.wilcoxon(nlgz_Jul_2014['nlgz_difference'], zero_method='pratt'))
print(str('Aug nlgz:'))
print(stats.wilcoxon(nlgz_Aug_2014['nlgz_difference'], zero_method='pratt'))   
print(str('Sep nlgz:'))
print(stats.wilcoxon(nlgz_Sep_2014['nlgz_difference'], zero_method='pratt'))    
print(str('Oct nlgz:'))
print(stats.wilcoxon(nlgz_Oct_2014['nlgz_difference'], zero_method='pratt'))
print(str('Nov nlgz:'))
print(stats.wilcoxon(nlgz_Nov_2014['nlgz_difference'], zero_method='pratt'))
print(str('Dec nlgz:'))
print(stats.wilcoxon(nlgz_Dec_2014['nlgz_difference'], zero_method='pratt'))

Jan nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Feb nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Mar nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Apr nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
May nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Jun nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Jul nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Aug nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Sep nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Oct nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Nov nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)
Dec nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)


#### Looking at seasonal differences

In [93]:
nlgz_dry_2013= [nlgz_May_2013,nlgz_Jun_2013,nlgz_Jul_2013,nlgz_Aug_2013,nlgz_Sep_2013]
nlgz_dry_2013 = pd.concat(nlgz_dry_2013, axis=0)
nlgz_dry_2014= [nlgz_May_2014,nlgz_Jun_2014,nlgz_Jul_2014,nlgz_Aug_2014,nlgz_Sep_2014]
nlgz_dry_2014 = pd.concat(nlgz_dry_2014, axis=0)

In [95]:
print(str('Dry season nlgz:'))
print(stats.wilcoxon(nlgz_dry_2014['nlgz_difference'], zero_method='pratt'))

Dry season nlgz:
WilcoxonResult(statistic=0.0, pvalue=0.0)


### Heatmaps

The Wilcoxon test so far indicates no clear differences within groups by month or season. I wonder if sectors may show something interesting...

In [105]:
nsmz_May_2013_NB= nsmz_May_2013.loc[nsmz_May_2013['sector']=='NB']
nsmz_May_2013_CB= nsmz_May_2013.loc[nsmz_May_2013['sector']=='CB']
nsmz_May_2013_SB= nsmz_May_2013.loc[nsmz_May_2013['sector']=='SB']

nsmz_Jun_2013_NB= nsmz_Jun_2013.loc[nsmz_Jun_2013['sector']=='NB']
nsmz_Jun_2013_CB= nsmz_Jun_2013.loc[nsmz_Jun_2013['sector']=='CB']
nsmz_Jun_2013_SB= nsmz_Jun_2013.loc[nsmz_Jun_2013['sector']=='SB']

nsmz_Jul_2013_NB= nsmz_Jul_2013.loc[nsmz_Jul_2013['sector']=='NB']
nsmz_Jul_2013_CB= nsmz_Jul_2013.loc[nsmz_Jul_2013['sector']=='CB']
nsmz_Jul_2013_SB= nsmz_Jul_2013.loc[nsmz_Jul_2013['sector']=='SB']

nsmz_Aug_2013_NB= nsmz_Aug_2013.loc[nsmz_Aug_2013['sector']=='NB']
nsmz_Aug_2013_CB= nsmz_Aug_2013.loc[nsmz_Aug_2013['sector']=='CB']
nsmz_Aug_2013_SB= nsmz_Aug_2013.loc[nsmz_Aug_2013['sector']=='SB']

nsmz_Sep_2013_NB= nsmz_Sep_2013.loc[nsmz_Sep_2013['sector']=='NB']
nsmz_Sep_2013_CB= nsmz_Sep_2013.loc[nsmz_Sep_2013['sector']=='CB']
nsmz_Sep_2013_SB= nsmz_Sep_2013.loc[nsmz_Sep_2013['sector']=='SB']

nmdz_May_2013_NB= nmdz_May_2013.loc[nmdz_May_2013['sector']=='NB']
nmdz_May_2013_CB= nmdz_May_2013.loc[nmdz_May_2013['sector']=='CB']
nmdz_May_2013_SB= nmdz_May_2013.loc[nmdz_May_2013['sector']=='SB']

nmdz_Jun_2013_NB= nmdz_Jun_2013.loc[nmdz_Jun_2013['sector']=='NB']
nmdz_Jun_2013_CB= nmdz_Jun_2013.loc[nmdz_Jun_2013['sector']=='CB']
nmdz_Jun_2013_SB= nmdz_Jun_2013.loc[nmdz_Jun_2013['sector']=='SB']

nmdz_Jul_2013_NB= nmdz_Jul_2013.loc[nmdz_Jul_2013['sector']=='NB']
nmdz_Jul_2013_CB= nmdz_Jul_2013.loc[nmdz_Jul_2013['sector']=='CB']
nmdz_Jul_2013_SB= nmdz_Jul_2013.loc[nmdz_Jul_2013['sector']=='SB']

nmdz_Aug_2013_NB= nmdz_Aug_2013.loc[nmdz_Aug_2013['sector']=='NB']
nmdz_Aug_2013_CB= nmdz_Aug_2013.loc[nmdz_Aug_2013['sector']=='CB']
nmdz_Aug_2013_SB= nmdz_Aug_2013.loc[nmdz_Aug_2013['sector']=='SB']

nmdz_Sep_2013_NB= nmdz_Sep_2013.loc[nmdz_Sep_2013['sector']=='NB']
nmdz_Sep_2013_CB= nmdz_Sep_2013.loc[nmdz_Sep_2013['sector']=='CB']
nmdz_Sep_2013_SB= nmdz_Sep_2013.loc[nmdz_Sep_2013['sector']=='SB']

nlgz_May_2013_NB= nlgz_May_2013.loc[nlgz_May_2013['sector']=='NB']
nlgz_May_2013_CB= nlgz_May_2013.loc[nlgz_May_2013['sector']=='CB']
nlgz_May_2013_SB= nlgz_May_2013.loc[nlgz_May_2013['sector']=='SB']

nlgz_Jun_2013_NB= nlgz_Jun_2013.loc[nlgz_Jun_2013['sector']=='NB']
nlgz_Jun_2013_CB= nlgz_Jun_2013.loc[nlgz_Jun_2013['sector']=='CB']
nlgz_Jun_2013_SB= nlgz_Jun_2013.loc[nlgz_Jun_2013['sector']=='SB']

nlgz_Jul_2013_NB= nlgz_Jul_2013.loc[nlgz_Jul_2013['sector']=='NB']
nlgz_Jul_2013_CB= nlgz_Jul_2013.loc[nlgz_Jul_2013['sector']=='CB']
nlgz_Jul_2013_SB= nlgz_Jul_2013.loc[nlgz_Jul_2013['sector']=='SB']

nlgz_Aug_2013_NB= nlgz_Aug_2013.loc[nlgz_Aug_2013['sector']=='NB']
nlgz_Aug_2013_CB= nlgz_Aug_2013.loc[nlgz_Aug_2013['sector']=='CB']
nlgz_Aug_2013_SB= nlgz_Aug_2013.loc[nlgz_Aug_2013['sector']=='SB']

nlgz_Sep_2013_NB= nlgz_Sep_2013.loc[nlgz_Sep_2013['sector']=='NB']
nlgz_Sep_2013_CB= nlgz_Sep_2013.loc[nlgz_Sep_2013['sector']=='CB']
nlgz_Sep_2013_SB= nlgz_Sep_2013.loc[nlgz_Sep_2013['sector']=='SB']
'''
nsm_May_2013_NB= nsm_May_2013.loc[nsm_May_2013['sector']=='NB']
nsm_May_2013_CB= nsm_May_2013.loc[nsm_May_2013['sector']=='CB']
nsm_May_2013_SB= nsm_May_2013.loc[nsm_May_2013['sector']=='SB']

nsm_Jun_2013_NB= nsm_Jun_2013.loc[nsm_Jun_2013['sector']=='NB']
nsm_Jun_2013_CB= nsm_Jun_2013.loc[nsm_Jun_2013['sector']=='CB']
nsm_Jun_2013_SB= nsm_Jun_2013.loc[nsm_Jun_2013['sector']=='SB']

nsm_Jul_2013_NB= nsm_Jul_2013.loc[nsm_Jul_2013['sector']=='NB']
nsm_Jul_2013_CB= nsm_Jul_2013.loc[nsm_Jul_2013['sector']=='CB']
nsm_Jul_2013_SB= nsm_Jul_2013.loc[nsm_Jul_2013['sector']=='SB']

nsm_Aug_2013_NB= nsm_Aug_2013.loc[nsm_Aug_2013['sector']=='NB']
nsm_Aug_2013_CB= nsm_Aug_2013.loc[nsm_Aug_2013['sector']=='CB']
nsm_Aug_2013_SB= nsm_Aug_2013.loc[nsm_Aug_2013['sector']=='SB']

nsm_Sep_2013_NB= nsm_Sep_2013.loc[nsm_Sep_2013['sector']=='NB']
nsm_Sep_2013_CB= nsm_Sep_2013.loc[nsm_Sep_2013['sector']=='CB']
nsm_Sep_2013_SB= nsm_Sep_2013.loc[nsm_Sep_2013['sector']=='SB']

nlg_May_2013_NB= nlg_May_2013.loc[nlg_May_2013['sector']=='NB']
nlg_May_2013_CB= nlg_May_2013.loc[nlg_May_2013['sector']=='CB']
nlg_May_2013_SB= nlg_May_2013.loc[nlg_May_2013['sector']=='SB']

nlg_Jun_2013_NB= nlg_Jun_2013.loc[nlg_Jun_2013['sector']=='NB']
nlg_Jun_2013_CB= nlg_Jun_2013.loc[nlg_Jun_2013['sector']=='CB']
nlg_Jun_2013_SB= nlg_Jun_2013.loc[nlg_Jun_2013['sector']=='SB']

nlg_Jul_2013_NB= nlg_Jul_2013.loc[nlg_Jul_2013['sector']=='NB']
nlg_Jul_2013_CB= nlg_Jul_2013.loc[nlg_Jul_2013['sector']=='CB']
nlg_Jul_2013_SB= nlg_Jul_2013.loc[nlg_Jul_2013['sector']=='SB']

nlg_Aug_2013_NB= nlg_Aug_2013.loc[nlg_Aug_2013['sector']=='NB']
nlg_Aug_2013_CB= nlg_Aug_2013.loc[nlg_Aug_2013['sector']=='CB']
nlg_Aug_2013_SB= nlg_Aug_2013.loc[nlg_Aug_2013['sector']=='SB']

nlg_Sep_2013_NB= nlg_Sep_2013.loc[nlg_Sep_2013['sector']=='NB']
nlg_Sep_2013_CB= nlg_Sep_2013.loc[nlg_Sep_2013['sector']=='CB']
nlg_Sep_2013_SB= nlg_Sep_2013.loc[nlg_Sep_2013['sector']=='SB']
'''

"\nnsm_May_2013_NB= nsm_May_2013.loc[nsm_May_2013['sector']=='NB']\nnsm_May_2013_CB= nsm_May_2013.loc[nsm_May_2013['sector']=='CB']\nnsm_May_2013_SB= nsm_May_2013.loc[nsm_May_2013['sector']=='SB']\n\nnsm_Jun_2013_NB= nsm_Jun_2013.loc[nsm_Jun_2013['sector']=='NB']\nnsm_Jun_2013_CB= nsm_Jun_2013.loc[nsm_Jun_2013['sector']=='CB']\nnsm_Jun_2013_SB= nsm_Jun_2013.loc[nsm_Jun_2013['sector']=='SB']\n\nnsm_Jul_2013_NB= nsm_Jul_2013.loc[nsm_Jul_2013['sector']=='NB']\nnsm_Jul_2013_CB= nsm_Jul_2013.loc[nsm_Jul_2013['sector']=='CB']\nnsm_Jul_2013_SB= nsm_Jul_2013.loc[nsm_Jul_2013['sector']=='SB']\n\nnsm_Aug_2013_NB= nsm_Aug_2013.loc[nsm_Aug_2013['sector']=='NB']\nnsm_Aug_2013_CB= nsm_Aug_2013.loc[nsm_Aug_2013['sector']=='CB']\nnsm_Aug_2013_SB= nsm_Aug_2013.loc[nsm_Aug_2013['sector']=='SB']\n\nnsm_Sep_2013_NB= nsm_Sep_2013.loc[nsm_Sep_2013['sector']=='NB']\nnsm_Sep_2013_CB= nsm_Sep_2013.loc[nsm_Sep_2013['sector']=='CB']\nnsm_Sep_2013_SB= nsm_Sep_2013.loc[nsm_Sep_2013['sector']=='SB']\n\nnlg_May_2013

In [106]:
print(str('Dry season median nsmz 2013 (NB):'))
print((nsmz_May_2013_NB.median()) + 
      (nsmz_Jun_2013_NB.median()) + 
       (nsmz_Jul_2013_NB.median()) + 
       (nsmz_Aug_2013_NB.median()) + 
       (nsmz_Sep_2013_NB.median()))

print(str('Dry season median nsmz 2013 (CB):'))
print((nsmz_May_2013_CB.median()) + 
      (nsmz_Jun_2013_CB.median()) + 
       (nsmz_Jul_2013_CB.median()) + 
       (nsmz_Aug_2013_CB.median()) + 
       (nsmz_Sep_2013_CB.median()))

print(str('Dry season median nsmz 2013 (SB):'))
print((nsmz_May_2013_SB.median()) + 
      (nsmz_Jun_2013_SB.median()) + 
       (nsmz_Jul_2013_SB.median()) + 
       (nsmz_Aug_2013_SB.median()) + 
       (nsmz_Sep_2013_SB.median()))

print(str('Dry season median nmdz 2013 (NB):'))
print((nmdz_May_2013_NB.median()) + 
      (nmdz_Jun_2013_NB.median()) + 
       (nmdz_Jul_2013_NB.median()) + 
       (nmdz_Aug_2013_NB.median()) + 
       (nmdz_Sep_2013_NB.median()))

print(str('Dry season median nmdz 2013 (CB):'))
print((nmdz_May_2013_CB.median()) + 
      (nmdz_Jun_2013_CB.median()) + 
       (nmdz_Jul_2013_CB.median()) + 
       (nmdz_Aug_2013_CB.median()) + 
       (nmdz_Sep_2013_CB.median()))

print(str('Dry season median nmdz 2013 (SB):'))
print((nmdz_May_2013_SB.median()) + 
      (nmdz_Jun_2013_SB.median()) + 
       (nmdz_Jul_2013_SB.median()) + 
       (nmdz_Aug_2013_SB.median()) + 
       (nmdz_Sep_2013_SB.median()))

print(str('Dry season median nlgz 2013 (NB):'))
print((nlgz_May_2013_NB.median()) + 
      (nlgz_Jun_2013_NB.median()) + 
       (nlgz_Jul_2013_NB.median()) + 
       (nlgz_Aug_2013_NB.median()) + 
       (nlgz_Sep_2013_NB.median()))

print(str('Dry season median nlgz 2013 (CB):'))
print((nlgz_May_2013_CB.median()) + 
      (nlgz_Jun_2013_CB.median()) + 
       (nlgz_Jul_2013_CB.median()) + 
       (nlgz_Aug_2013_CB.median()) + 
       (nlgz_Sep_2013_CB.median()))

print(str('Dry season median nlgz 2013 (SB):'))
print((nlgz_May_2013_SB.median()) + 
      (nlgz_Jun_2013_SB.median()) + 
       (nlgz_Jul_2013_SB.median()) + 
       (nlgz_Aug_2013_SB.median()) + 
       (nlgz_Sep_2013_SB.median()))

Dry season median nsmz 2013 (NB):
Unnamed: 0       2.001375e+05
concentration    4.678852e-07
year             1.006500e+04
dtype: float64
Dry season median nsmz 2013 (CB):
Unnamed: 0       2.127425e+05
concentration    5.340631e-07
year             1.006500e+04
dtype: float64
Dry season median nsmz 2013 (SB):
Unnamed: 0       2.233325e+05
concentration    6.603765e-07
year             1.006500e+04
dtype: float64
Dry season median nmdz 2013 (NB):
Unnamed: 0       2.001375e+05
concentration    3.112258e-07
year             1.006500e+04
dtype: float64
Dry season median nmdz 2013 (CB):
Unnamed: 0       2.127425e+05
concentration    4.170460e-07
year             1.006500e+04
dtype: float64
Dry season median nmdz 2013 (SB):
Unnamed: 0       2.233325e+05
concentration    6.245155e-07
year             1.006500e+04
dtype: float64
Dry season median nlgz 2013 (NB):
Unnamed: 0       2.001375e+05
concentration    5.986258e-08
year             1.006500e+04
dtype: float64
Dry season median nlgz 2013

In [108]:
nsmz_May_2014_NB= nsmz_May_2014.loc[nsmz_May_2014['sector']=='NB']
nsmz_May_2014_CB= nsmz_May_2014.loc[nsmz_May_2014['sector']=='CB']
nsmz_May_2014_SB= nsmz_May_2014.loc[nsmz_May_2014['sector']=='SB']

nsmz_Jun_2014_NB= nsmz_Jun_2014.loc[nsmz_Jun_2014['sector']=='NB']
nsmz_Jun_2014_CB= nsmz_Jun_2014.loc[nsmz_Jun_2014['sector']=='CB']
nsmz_Jun_2014_SB= nsmz_Jun_2014.loc[nsmz_Jun_2014['sector']=='SB']

nsmz_Jul_2014_NB= nsmz_Jul_2014.loc[nsmz_Jul_2014['sector']=='NB']
nsmz_Jul_2014_CB= nsmz_Jul_2014.loc[nsmz_Jul_2014['sector']=='CB']
nsmz_Jul_2014_SB= nsmz_Jul_2014.loc[nsmz_Jul_2014['sector']=='SB']

nsmz_Aug_2014_NB= nsmz_Aug_2014.loc[nsmz_Aug_2014['sector']=='NB']
nsmz_Aug_2014_CB= nsmz_Aug_2014.loc[nsmz_Aug_2014['sector']=='CB']
nsmz_Aug_2014_SB= nsmz_Aug_2014.loc[nsmz_Aug_2014['sector']=='SB']

nsmz_Sep_2014_NB= nsmz_Sep_2014.loc[nsmz_Sep_2014['sector']=='NB']
nsmz_Sep_2014_CB= nsmz_Sep_2014.loc[nsmz_Sep_2014['sector']=='CB']
nsmz_Sep_2014_SB= nsmz_Sep_2014.loc[nsmz_Sep_2014['sector']=='SB']

nmdz_May_2014_NB= nmdz_May_2014.loc[nmdz_May_2014['sector']=='NB']
nmdz_May_2014_CB= nmdz_May_2014.loc[nmdz_May_2014['sector']=='CB']
nmdz_May_2014_SB= nmdz_May_2014.loc[nmdz_May_2014['sector']=='SB']

nmdz_Jun_2014_NB= nmdz_Jun_2014.loc[nmdz_Jun_2014['sector']=='NB']
nmdz_Jun_2014_CB= nmdz_Jun_2014.loc[nmdz_Jun_2014['sector']=='CB']
nmdz_Jun_2014_SB= nmdz_Jun_2014.loc[nmdz_Jun_2014['sector']=='SB']

nmdz_Jul_2014_NB= nmdz_Jul_2014.loc[nmdz_Jul_2014['sector']=='NB']
nmdz_Jul_2014_CB= nmdz_Jul_2014.loc[nmdz_Jul_2014['sector']=='CB']
nmdz_Jul_2014_SB= nmdz_Jul_2014.loc[nmdz_Jul_2014['sector']=='SB']

nmdz_Aug_2014_NB= nmdz_Aug_2014.loc[nmdz_Aug_2014['sector']=='NB']
nmdz_Aug_2014_CB= nmdz_Aug_2014.loc[nmdz_Aug_2014['sector']=='CB']
nmdz_Aug_2014_SB= nmdz_Aug_2014.loc[nmdz_Aug_2014['sector']=='SB']

nmdz_Sep_2014_NB= nmdz_Sep_2014.loc[nmdz_Sep_2014['sector']=='NB']
nmdz_Sep_2014_CB= nmdz_Sep_2014.loc[nmdz_Sep_2014['sector']=='CB']
nmdz_Sep_2014_SB= nmdz_Sep_2014.loc[nmdz_Sep_2014['sector']=='SB']

nlgz_May_2014_NB= nlgz_May_2014.loc[nlgz_May_2014['sector']=='NB']
nlgz_May_2014_CB= nlgz_May_2014.loc[nlgz_May_2014['sector']=='CB']
nlgz_May_2014_SB= nlgz_May_2014.loc[nlgz_May_2014['sector']=='SB']

nlgz_Jun_2014_NB= nlgz_Jun_2014.loc[nlgz_Jun_2014['sector']=='NB']
nlgz_Jun_2014_CB= nlgz_Jun_2014.loc[nlgz_Jun_2014['sector']=='CB']
nlgz_Jun_2014_SB= nlgz_Jun_2014.loc[nlgz_Jun_2014['sector']=='SB']

nlgz_Jul_2014_NB= nlgz_Jul_2014.loc[nlgz_Jul_2014['sector']=='NB']
nlgz_Jul_2014_CB= nlgz_Jul_2014.loc[nlgz_Jul_2014['sector']=='CB']
nlgz_Jul_2014_SB= nlgz_Jul_2014.loc[nlgz_Jul_2014['sector']=='SB']

nlgz_Aug_2014_NB= nlgz_Aug_2014.loc[nlgz_Aug_2014['sector']=='NB']
nlgz_Aug_2014_CB= nlgz_Aug_2014.loc[nlgz_Aug_2014['sector']=='CB']
nlgz_Aug_2014_SB= nlgz_Aug_2014.loc[nlgz_Aug_2014['sector']=='SB']

nlgz_Sep_2014_NB= nlgz_Sep_2014.loc[nlgz_Sep_2014['sector']=='NB']
nlgz_Sep_2014_CB= nlgz_Sep_2014.loc[nlgz_Sep_2014['sector']=='CB']
nlgz_Sep_2014_SB= nlgz_Sep_2014.loc[nlgz_Sep_2014['sector']=='SB']
'''
nsm_May_2014_NB= nsm_May_2014.loc[nsm_May_2014['sector']=='NB']
nsm_May_2014_CB= nsm_May_2014.loc[nsm_May_2014['sector']=='CB']
nsm_May_2014_SB= nsm_May_2014.loc[nsm_May_2014['sector']=='SB']

nsm_Jun_2014_NB= nsm_Jun_2014.loc[nsm_Jun_2014['sector']=='NB']
nsm_Jun_2014_CB= nsm_Jun_2014.loc[nsm_Jun_2014['sector']=='CB']
nsm_Jun_2014_SB= nsm_Jun_2014.loc[nsm_Jun_2014['sector']=='SB']

nsm_Jul_2014_NB= nsm_Jul_2014.loc[nsm_Jul_2014['sector']=='NB']
nsm_Jul_2014_CB= nsm_Jul_2014.loc[nsm_Jul_2014['sector']=='CB']
nsm_Jul_2014_SB= nsm_Jul_2014.loc[nsm_Jul_2014['sector']=='SB']

nsm_Aug_2014_NB= nsm_Aug_2014.loc[nsm_Aug_2014['sector']=='NB']
nsm_Aug_2014_CB= nsm_Aug_2014.loc[nsm_Aug_2014['sector']=='CB']
nsm_Aug_2014_SB= nsm_Aug_2014.loc[nsm_Aug_2014['sector']=='SB']

nsm_Sep_2014_NB= nsm_Sep_2014.loc[nsm_Sep_2014['sector']=='NB']
nsm_Sep_2014_CB= nsm_Sep_2014.loc[nsm_Sep_2014['sector']=='CB']
nsm_Sep_2014_SB= nsm_Sep_2014.loc[nsm_Sep_2014['sector']=='SB']

nlg_May_2014_NB= nlg_May_2014.loc[nlg_May_2014['sector']=='NB']
nlg_May_2014_CB= nlg_May_2014.loc[nlg_May_2014['sector']=='CB']
nlg_May_2014_SB= nlg_May_2014.loc[nlg_May_2014['sector']=='SB']

nlg_Jun_2014_NB= nlg_Jun_2014.loc[nlg_Jun_2014['sector']=='NB']
nlg_Jun_2014_CB= nlg_Jun_2014.loc[nlg_Jun_2014['sector']=='CB']
nlg_Jun_2014_SB= nlg_Jun_2014.loc[nlg_Jun_2014['sector']=='SB']

nlg_Jul_2014_NB= nlg_Jul_2014.loc[nlg_Jul_2014['sector']=='NB']
nlg_Jul_2014_CB= nlg_Jul_2014.loc[nlg_Jul_2014['sector']=='CB']
nlg_Jul_2014_SB= nlg_Jul_2014.loc[nlg_Jul_2014['sector']=='SB']

nlg_Aug_2014_NB= nlg_Aug_2014.loc[nlg_Aug_2014['sector']=='NB']
nlg_Aug_2014_CB= nlg_Aug_2014.loc[nlg_Aug_2014['sector']=='CB']
nlg_Aug_2014_SB= nlg_Aug_2014.loc[nlg_Aug_2014['sector']=='SB']

nlg_Sep_2014_NB= nlg_Sep_2014.loc[nlg_Sep_2014['sector']=='NB']
nlg_Sep_2014_CB= nlg_Sep_2014.loc[nlg_Sep_2014['sector']=='CB']
nlg_Sep_2014_SB= nlg_Sep_2014.loc[nlg_Sep_2014['sector']=='SB']
'''

"\nnsm_May_2014_NB= nsm_May_2014.loc[nsm_May_2014['sector']=='NB']\nnsm_May_2014_CB= nsm_May_2014.loc[nsm_May_2014['sector']=='CB']\nnsm_May_2014_SB= nsm_May_2014.loc[nsm_May_2014['sector']=='SB']\n\nnsm_Jun_2014_NB= nsm_Jun_2014.loc[nsm_Jun_2014['sector']=='NB']\nnsm_Jun_2014_CB= nsm_Jun_2014.loc[nsm_Jun_2014['sector']=='CB']\nnsm_Jun_2014_SB= nsm_Jun_2014.loc[nsm_Jun_2014['sector']=='SB']\n\nnsm_Jul_2014_NB= nsm_Jul_2014.loc[nsm_Jul_2014['sector']=='NB']\nnsm_Jul_2014_CB= nsm_Jul_2014.loc[nsm_Jul_2014['sector']=='CB']\nnsm_Jul_2014_SB= nsm_Jul_2014.loc[nsm_Jul_2014['sector']=='SB']\n\nnsm_Aug_2014_NB= nsm_Aug_2014.loc[nsm_Aug_2014['sector']=='NB']\nnsm_Aug_2014_CB= nsm_Aug_2014.loc[nsm_Aug_2014['sector']=='CB']\nnsm_Aug_2014_SB= nsm_Aug_2014.loc[nsm_Aug_2014['sector']=='SB']\n\nnsm_Sep_2014_NB= nsm_Sep_2014.loc[nsm_Sep_2014['sector']=='NB']\nnsm_Sep_2014_CB= nsm_Sep_2014.loc[nsm_Sep_2014['sector']=='CB']\nnsm_Sep_2014_SB= nsm_Sep_2014.loc[nsm_Sep_2014['sector']=='SB']\n\nnlg_May_2014

In [109]:
print(str('Dry season median nsmz 2014 (NB):'))
print((nsmz_May_2014_NB.median()) + 
      (nsmz_Jun_2014_NB.median()) + 
       (nsmz_Jul_2014_NB.median()) + 
       (nsmz_Aug_2014_NB.median()) + 
       (nsmz_Sep_2014_NB.median()))

print(str('Dry season median nsmz 2014 (CB):'))
print((nsmz_May_2014_CB.median()) + 
      (nsmz_Jun_2014_CB.median()) + 
       (nsmz_Jul_2014_CB.median()) + 
       (nsmz_Aug_2014_CB.median()) + 
       (nsmz_Sep_2014_CB.median()))

print(str('Dry season median nsmz 2014 (SB):'))
print((nsmz_May_2014_SB.median()) + 
      (nsmz_Jun_2014_SB.median()) + 
       (nsmz_Jul_2014_SB.median()) + 
       (nsmz_Aug_2014_SB.median()) + 
       (nsmz_Sep_2014_SB.median()))

print(str('Dry season median nmdz 2014 (NB):'))
print((nmdz_May_2014_NB.median()) + 
      (nmdz_Jun_2014_NB.median()) + 
       (nmdz_Jul_2014_NB.median()) + 
       (nmdz_Aug_2014_NB.median()) + 
       (nmdz_Sep_2014_NB.median()))

print(str('Dry season median nmdz 2014 (CB):'))
print((nmdz_May_2014_CB.median()) + 
      (nmdz_Jun_2014_CB.median()) + 
       (nmdz_Jul_2014_CB.median()) + 
       (nmdz_Aug_2014_CB.median()) + 
       (nmdz_Sep_2014_CB.median()))

print(str('Dry season median nmdz 2014 (SB):'))
print((nmdz_May_2014_SB.median()) + 
      (nmdz_Jun_2014_SB.median()) + 
       (nmdz_Jul_2014_SB.median()) + 
       (nmdz_Aug_2014_SB.median()) + 
       (nmdz_Sep_2014_SB.median()))

print(str('Dry season median nlgz 2014 (NB):'))
print((nlgz_May_2014_NB.median()) + 
      (nlgz_Jun_2014_NB.median()) + 
       (nlgz_Jul_2014_NB.median()) + 
       (nlgz_Aug_2014_NB.median()) + 
       (nlgz_Sep_2014_NB.median()))

print(str('Dry season median nlgz 2014 (CB):'))
print((nlgz_May_2014_CB.median()) + 
      (nlgz_Jun_2014_CB.median()) + 
       (nlgz_Jul_2014_CB.median()) + 
       (nlgz_Aug_2014_CB.median()) + 
       (nlgz_Sep_2014_CB.median()))

print(str('Dry season median nlgz 2014 (SB):'))
print((nlgz_May_2014_SB.median()) + 
      (nlgz_Jun_2014_SB.median()) + 
       (nlgz_Jul_2014_SB.median()) + 
       (nlgz_Aug_2014_SB.median()) + 
       (nlgz_Sep_2014_SB.median()))

Dry season median nsmz 2014 (NB):
Unnamed: 0         2.001375e+05
concentration      4.843919e-07
year               1.007000e+04
nsmz_difference    4.188630e-08
dtype: float64
Dry season median nsmz 2014 (CB):
Unnamed: 0         2.127425e+05
concentration      5.681661e-07
year               1.007000e+04
nsmz_difference    3.930785e-08
dtype: float64
Dry season median nsmz 2014 (SB):
Unnamed: 0         2.233325e+05
concentration      6.630281e-07
year               1.007000e+04
nsmz_difference    2.446531e-08
dtype: float64
Dry season median nmdz 2014 (NB):
Unnamed: 0         2.001375e+05
concentration      3.255327e-07
year               1.007000e+04
nmdz_difference    1.720501e-08
dtype: float64
Dry season median nmdz 2014 (CB):
Unnamed: 0         2.127425e+05
concentration      4.477446e-07
year               1.007000e+04
nmdz_difference    3.667747e-08
dtype: float64
Dry season median nmdz 2014 (SB):
Unnamed: 0         2.233325e+05
concentration      6.252172e-07
year             

In [126]:
# Dry season

fig = go.Figure(data=go.Heatmap(
                   z=[[4.84/4.67, 5.68/5.34, 6.63/6.60], 
                      [3.26/3.11, 4.48/4.17, 6.25/6.24], 
                      [.897/.599,1.06/.725,1.35/1.03]],
                   x=['NB', 'CB', 'SB'],
                   y=['Small zoos', 'Med zoos', 'Large zoos'],
                    colorscale="Viridis",
                    zmin=1, zmax=1.5))                  

fig.update_layout(
    title='Dry (May-Sep) season proportional differences: 2014 v. 2013')

fig.show()

In [113]:
nsmz_Jan_2013_NB= nsmz_Jan_2013.loc[nsmz_Jan_2013['sector']=='NB']
nsmz_Jan_2013_CB= nsmz_Jan_2013.loc[nsmz_Jan_2013['sector']=='CB']
nsmz_Jan_2013_SB= nsmz_Jan_2013.loc[nsmz_Jan_2013['sector']=='SB']

nsmz_Feb_2013_NB= nsmz_Feb_2013.loc[nsmz_Feb_2013['sector']=='NB']
nsmz_Feb_2013_CB= nsmz_Feb_2013.loc[nsmz_Feb_2013['sector']=='CB']
nsmz_Feb_2013_SB= nsmz_Feb_2013.loc[nsmz_Feb_2013['sector']=='SB']

nsmz_Mar_2013_NB= nsmz_Mar_2013.loc[nsmz_Mar_2013['sector']=='NB']
nsmz_Mar_2013_CB= nsmz_Mar_2013.loc[nsmz_Mar_2013['sector']=='CB']
nsmz_Mar_2013_SB= nsmz_Mar_2013.loc[nsmz_Mar_2013['sector']=='SB']

nsmz_Apr_2013_NB= nsmz_Apr_2013.loc[nsmz_Apr_2013['sector']=='NB']
nsmz_Apr_2013_CB= nsmz_Apr_2013.loc[nsmz_Apr_2013['sector']=='CB']
nsmz_Apr_2013_SB= nsmz_Apr_2013.loc[nsmz_Apr_2013['sector']=='SB']

nsmz_Oct_2013_NB= nsmz_Oct_2013.loc[nsmz_Oct_2013['sector']=='NB']
nsmz_Oct_2013_CB= nsmz_Oct_2013.loc[nsmz_Oct_2013['sector']=='CB']
nsmz_Oct_2013_SB= nsmz_Oct_2013.loc[nsmz_Oct_2013['sector']=='SB']

nsmz_Nov_2013_NB= nsmz_Nov_2013.loc[nsmz_Nov_2013['sector']=='NB']
nsmz_Nov_2013_CB= nsmz_Nov_2013.loc[nsmz_Nov_2013['sector']=='CB']
nsmz_Nov_2013_SB= nsmz_Nov_2013.loc[nsmz_Nov_2013['sector']=='SB']

nsmz_Dec_2013_NB= nsmz_Dec_2013.loc[nsmz_Dec_2013['sector']=='NB']
nsmz_Dec_2013_CB= nsmz_Dec_2013.loc[nsmz_Dec_2013['sector']=='CB']
nsmz_Dec_2013_SB= nsmz_Dec_2013.loc[nsmz_Dec_2013['sector']=='SB']


nmdz_Jan_2013_NB= nmdz_Jan_2013.loc[nmdz_Jan_2013['sector']=='NB']
nmdz_Jan_2013_CB= nmdz_Jan_2013.loc[nmdz_Jan_2013['sector']=='CB']
nmdz_Jan_2013_SB= nmdz_Jan_2013.loc[nmdz_Jan_2013['sector']=='SB']

nmdz_Feb_2013_NB= nmdz_Feb_2013.loc[nmdz_Feb_2013['sector']=='NB']
nmdz_Feb_2013_CB= nmdz_Feb_2013.loc[nmdz_Feb_2013['sector']=='CB']
nmdz_Feb_2013_SB= nmdz_Feb_2013.loc[nmdz_Feb_2013['sector']=='SB']

nmdz_Mar_2013_NB= nmdz_Mar_2013.loc[nmdz_Mar_2013['sector']=='NB']
nmdz_Mar_2013_CB= nmdz_Mar_2013.loc[nmdz_Mar_2013['sector']=='CB']
nmdz_Mar_2013_SB= nmdz_Mar_2013.loc[nmdz_Mar_2013['sector']=='SB']

nmdz_Apr_2013_NB= nmdz_Apr_2013.loc[nmdz_Apr_2013['sector']=='NB']
nmdz_Apr_2013_CB= nmdz_Apr_2013.loc[nmdz_Apr_2013['sector']=='CB']
nmdz_Apr_2013_SB= nmdz_Apr_2013.loc[nmdz_Apr_2013['sector']=='SB']

nmdz_Oct_2013_NB= nmdz_Oct_2013.loc[nmdz_Oct_2013['sector']=='NB']
nmdz_Oct_2013_CB= nmdz_Oct_2013.loc[nmdz_Oct_2013['sector']=='CB']
nmdz_Oct_2013_SB= nmdz_Oct_2013.loc[nmdz_Oct_2013['sector']=='SB']

nmdz_Nov_2013_NB= nmdz_Nov_2013.loc[nmdz_Nov_2013['sector']=='NB']
nmdz_Nov_2013_CB= nmdz_Nov_2013.loc[nmdz_Nov_2013['sector']=='CB']
nmdz_Nov_2013_SB= nmdz_Nov_2013.loc[nmdz_Nov_2013['sector']=='SB']

nmdz_Dec_2013_NB= nmdz_Dec_2013.loc[nmdz_Dec_2013['sector']=='NB']
nmdz_Dec_2013_CB= nmdz_Dec_2013.loc[nmdz_Dec_2013['sector']=='CB']
nmdz_Dec_2013_SB= nmdz_Dec_2013.loc[nmdz_Dec_2013['sector']=='SB']

nlgz_Jan_2013_NB= nlgz_Jan_2013.loc[nlgz_Jan_2013['sector']=='NB']
nlgz_Jan_2013_CB= nlgz_Jan_2013.loc[nlgz_Jan_2013['sector']=='CB']
nlgz_Jan_2013_SB= nlgz_Jan_2013.loc[nlgz_Jan_2013['sector']=='SB']

nlgz_Feb_2013_NB= nlgz_Feb_2013.loc[nlgz_Feb_2013['sector']=='NB']
nlgz_Feb_2013_CB= nlgz_Feb_2013.loc[nlgz_Feb_2013['sector']=='CB']
nlgz_Feb_2013_SB= nlgz_Feb_2013.loc[nlgz_Feb_2013['sector']=='SB']

nlgz_Mar_2013_NB= nlgz_Mar_2013.loc[nlgz_Mar_2013['sector']=='NB']
nlgz_Mar_2013_CB= nlgz_Mar_2013.loc[nlgz_Mar_2013['sector']=='CB']
nlgz_Mar_2013_SB= nlgz_Mar_2013.loc[nlgz_Mar_2013['sector']=='SB']

nlgz_Apr_2013_NB= nlgz_Apr_2013.loc[nlgz_Apr_2013['sector']=='NB']
nlgz_Apr_2013_CB= nlgz_Apr_2013.loc[nlgz_Apr_2013['sector']=='CB']
nlgz_Apr_2013_SB= nlgz_Apr_2013.loc[nlgz_Apr_2013['sector']=='SB']

nlgz_Oct_2013_NB= nlgz_Oct_2013.loc[nlgz_Oct_2013['sector']=='NB']
nlgz_Oct_2013_CB= nlgz_Oct_2013.loc[nlgz_Oct_2013['sector']=='CB']
nlgz_Oct_2013_SB= nlgz_Oct_2013.loc[nlgz_Oct_2013['sector']=='SB']

nlgz_Nov_2013_NB= nlgz_Nov_2013.loc[nlgz_Nov_2013['sector']=='NB']
nlgz_Nov_2013_CB= nlgz_Nov_2013.loc[nlgz_Nov_2013['sector']=='CB']
nlgz_Nov_2013_SB= nlgz_Nov_2013.loc[nlgz_Nov_2013['sector']=='SB']

nlgz_Dec_2013_NB= nlgz_Dec_2013.loc[nlgz_Dec_2013['sector']=='NB']
nlgz_Dec_2013_CB= nlgz_Dec_2013.loc[nlgz_Dec_2013['sector']=='CB']
nlgz_Dec_2013_SB= nlgz_Dec_2013.loc[nlgz_Dec_2013['sector']=='SB']

In [115]:
print(str('Wet season median nsmz 2013 (NB):'))
print((nsmz_Jan_2013_NB.median()) + 
      (nsmz_Feb_2013_NB.median()) + 
       (nsmz_Mar_2013_NB.median()) + 
       (nsmz_Apr_2013_NB.median()) + 
       (nsmz_Oct_2013_NB.median()) +
       (nsmz_Nov_2013_NB.median()) +  
       (nsmz_Dec_2013_NB.median()))

print(str('Wet season median nsmz 2013 (CB):'))
print((nsmz_Jan_2013_CB.median()) + 
      (nsmz_Feb_2013_CB.median()) + 
       (nsmz_Mar_2013_CB.median()) + 
       (nsmz_Apr_2013_CB.median()) + 
       (nsmz_Oct_2013_CB.median()) + 
       (nsmz_Nov_2013_CB.median()) + 
       (nsmz_Dec_2013_CB.median()))

print(str('Wet season median nsmz 2013 (SB):'))
print((nsmz_Jan_2013_SB.median()) + 
      (nsmz_Feb_2013_SB.median()) + 
       (nsmz_Mar_2013_SB.median()) + 
       (nsmz_Apr_2013_SB.median()) +
       (nsmz_Oct_2013_SB.median()) +
       (nsmz_Nov_2013_SB.median()) + 
       (nsmz_Dec_2013_SB.median()))

print(str('Wet season median nmdz 2013 (NB):'))
print((nmdz_Jan_2013_NB.median()) + 
      (nmdz_Feb_2013_NB.median()) + 
       (nmdz_Mar_2013_NB.median()) + 
       (nmdz_Apr_2013_NB.median()) +
       (nmdz_Oct_2013_NB.median()) +
       (nmdz_Nov_2013_NB.median()) + 
       (nmdz_Dec_2013_NB.median()))

print(str('Wet season median nmdz 2013 (CB):'))
print((nmdz_Jan_2013_CB.median()) + 
      (nmdz_Feb_2013_CB.median()) + 
       (nmdz_Mar_2013_CB.median()) + 
       (nmdz_Apr_2013_CB.median()) +
       (nmdz_Oct_2013_CB.median()) +
       (nmdz_Nov_2013_CB.median()) + 
       (nmdz_Dec_2013_CB.median()))

print(str('Wet season median nmdz 2013 (SB):'))
print((nmdz_Jan_2013_SB.median()) + 
      (nmdz_Feb_2013_SB.median()) + 
       (nmdz_Mar_2013_SB.median()) + 
       (nmdz_Apr_2013_SB.median()) + 
       (nmdz_Oct_2013_SB.median()) + 
       (nmdz_Nov_2013_SB.median()) + 
       (nmdz_Dec_2013_SB.median()))

print(str('Wet season median nlgz 2013 (NB):'))
print((nlgz_Jan_2013_NB.median()) + 
      (nlgz_Feb_2013_NB.median()) + 
       (nlgz_Mar_2013_NB.median()) + 
       (nlgz_Apr_2013_NB.median()) +
       (nlgz_Oct_2013_NB.median()) +
       (nlgz_Nov_2013_NB.median()) + 
       (nlgz_Dec_2013_NB.median()))

print(str('Wet season median nlgz 2013 (CB):'))
print((nlgz_Jan_2013_CB.median()) + 
      (nlgz_Feb_2013_CB.median()) + 
       (nlgz_Mar_2013_CB.median()) + 
       (nlgz_Apr_2013_CB.median()) +
       (nlgz_Oct_2013_CB.median()) +
       (nlgz_Nov_2013_CB.median()) +
       (nlgz_Dec_2013_CB.median()))

print(str('Wet season median nlgz 2013 (SB):'))
print((nlgz_Jan_2013_SB.median()) + 
      (nlgz_Feb_2013_SB.median()) + 
       (nlgz_Mar_2013_SB.median()) + 
       (nlgz_Apr_2013_SB.median()) +
       (nlgz_Oct_2013_SB.median()) +
       (nlgz_Nov_2013_SB.median()) + 
       (nlgz_Dec_2013_SB.median()))

Wet season median nsmz 2013 (NB):
Unnamed: 0       2.412885e+05
concentration    7.899343e-07
year             1.409100e+04
dtype: float64
Wet season median nsmz 2013 (CB):
Unnamed: 0       2.589355e+05
concentration    8.658717e-07
year             1.409100e+04
dtype: float64
Wet season median nsmz 2013 (SB):
Unnamed: 0       2.737615e+05
concentration    8.914081e-07
year             1.409100e+04
dtype: float64
Wet season median nmdz 2013 (NB):
Unnamed: 0       2.412885e+05
concentration    4.650950e-07
year             1.409100e+04
dtype: float64
Wet season median nmdz 2013 (CB):
Unnamed: 0       2.589355e+05
concentration    6.039056e-07
year             1.409100e+04
dtype: float64
Wet season median nmdz 2013 (SB):
Unnamed: 0       2.737615e+05
concentration    8.383105e-07
year             1.409100e+04
dtype: float64
Wet season median nlgz 2013 (NB):
Unnamed: 0       2.412885e+05
concentration    1.004453e-07
year             1.409100e+04
dtype: float64
Wet season median nlgz 2013

In [116]:
nsmz_Jan_2014_NB= nsmz_Jan_2014.loc[nsmz_Jan_2014['sector']=='NB']
nsmz_Jan_2014_CB= nsmz_Jan_2014.loc[nsmz_Jan_2014['sector']=='CB']
nsmz_Jan_2014_SB= nsmz_Jan_2014.loc[nsmz_Jan_2014['sector']=='SB']

nsmz_Feb_2014_NB= nsmz_Feb_2014.loc[nsmz_Feb_2014['sector']=='NB']
nsmz_Feb_2014_CB= nsmz_Feb_2014.loc[nsmz_Feb_2014['sector']=='CB']
nsmz_Feb_2014_SB= nsmz_Feb_2014.loc[nsmz_Feb_2014['sector']=='SB']

nsmz_Mar_2014_NB= nsmz_Mar_2014.loc[nsmz_Mar_2014['sector']=='NB']
nsmz_Mar_2014_CB= nsmz_Mar_2014.loc[nsmz_Mar_2014['sector']=='CB']
nsmz_Mar_2014_SB= nsmz_Mar_2014.loc[nsmz_Mar_2014['sector']=='SB']

nsmz_Apr_2014_NB= nsmz_Apr_2014.loc[nsmz_Apr_2014['sector']=='NB']
nsmz_Apr_2014_CB= nsmz_Apr_2014.loc[nsmz_Apr_2014['sector']=='CB']
nsmz_Apr_2014_SB= nsmz_Apr_2014.loc[nsmz_Apr_2014['sector']=='SB']

nsmz_Oct_2014_NB= nsmz_Oct_2014.loc[nsmz_Oct_2014['sector']=='NB']
nsmz_Oct_2014_CB= nsmz_Oct_2014.loc[nsmz_Oct_2014['sector']=='CB']
nsmz_Oct_2014_SB= nsmz_Oct_2014.loc[nsmz_Oct_2014['sector']=='SB']

nsmz_Nov_2014_NB= nsmz_Nov_2014.loc[nsmz_Nov_2014['sector']=='NB']
nsmz_Nov_2014_CB= nsmz_Nov_2014.loc[nsmz_Nov_2014['sector']=='CB']
nsmz_Nov_2014_SB= nsmz_Nov_2014.loc[nsmz_Nov_2014['sector']=='SB']

nsmz_Dec_2014_NB= nsmz_Dec_2014.loc[nsmz_Dec_2014['sector']=='NB']
nsmz_Dec_2014_CB= nsmz_Dec_2014.loc[nsmz_Dec_2014['sector']=='CB']
nsmz_Dec_2014_SB= nsmz_Dec_2014.loc[nsmz_Dec_2014['sector']=='SB']

nmdz_Jan_2014_NB= nmdz_Jan_2014.loc[nmdz_Jan_2014['sector']=='NB']
nmdz_Jan_2014_CB= nmdz_Jan_2014.loc[nmdz_Jan_2014['sector']=='CB']
nmdz_Jan_2014_SB= nmdz_Jan_2014.loc[nmdz_Jan_2014['sector']=='SB']

nmdz_Feb_2014_NB= nmdz_Feb_2014.loc[nmdz_Feb_2014['sector']=='NB']
nmdz_Feb_2014_CB= nmdz_Feb_2014.loc[nmdz_Feb_2014['sector']=='CB']
nmdz_Feb_2014_SB= nmdz_Feb_2014.loc[nmdz_Feb_2014['sector']=='SB']

nmdz_Mar_2014_NB= nmdz_Mar_2014.loc[nmdz_Mar_2014['sector']=='NB']
nmdz_Mar_2014_CB= nmdz_Mar_2014.loc[nmdz_Mar_2014['sector']=='CB']
nmdz_Mar_2014_SB= nmdz_Mar_2014.loc[nmdz_Mar_2014['sector']=='SB']

nmdz_Apr_2014_NB= nmdz_Apr_2014.loc[nmdz_Apr_2014['sector']=='NB']
nmdz_Apr_2014_CB= nmdz_Apr_2014.loc[nmdz_Apr_2014['sector']=='CB']
nmdz_Apr_2014_SB= nmdz_Apr_2014.loc[nmdz_Apr_2014['sector']=='SB']

nmdz_Oct_2014_NB= nmdz_Oct_2014.loc[nmdz_Oct_2014['sector']=='NB']
nmdz_Oct_2014_CB= nmdz_Oct_2014.loc[nmdz_Oct_2014['sector']=='CB']
nmdz_Oct_2014_SB= nmdz_Oct_2014.loc[nmdz_Oct_2014['sector']=='SB']

nmdz_Nov_2014_NB= nmdz_Nov_2014.loc[nmdz_Nov_2014['sector']=='NB']
nmdz_Nov_2014_CB= nmdz_Nov_2014.loc[nmdz_Nov_2014['sector']=='CB']
nmdz_Nov_2014_SB= nmdz_Nov_2014.loc[nmdz_Nov_2014['sector']=='SB']

nmdz_Dec_2014_NB= nmdz_Dec_2014.loc[nmdz_Dec_2014['sector']=='NB']
nmdz_Dec_2014_CB= nmdz_Dec_2014.loc[nmdz_Dec_2014['sector']=='CB']
nmdz_Dec_2014_SB= nmdz_Dec_2014.loc[nmdz_Dec_2014['sector']=='SB']

nlgz_Jan_2014_NB= nlgz_Jan_2014.loc[nlgz_Jan_2014['sector']=='NB']
nlgz_Jan_2014_CB= nlgz_Jan_2014.loc[nlgz_Jan_2014['sector']=='CB']
nlgz_Jan_2014_SB= nlgz_Jan_2014.loc[nlgz_Jan_2014['sector']=='SB']

nlgz_Feb_2014_NB= nlgz_Feb_2014.loc[nlgz_Feb_2014['sector']=='NB']
nlgz_Feb_2014_CB= nlgz_Feb_2014.loc[nlgz_Feb_2014['sector']=='CB']
nlgz_Feb_2014_SB= nlgz_Feb_2014.loc[nlgz_Feb_2014['sector']=='SB']

nlgz_Mar_2014_NB= nlgz_Mar_2014.loc[nlgz_Mar_2014['sector']=='NB']
nlgz_Mar_2014_CB= nlgz_Mar_2014.loc[nlgz_Mar_2014['sector']=='CB']
nlgz_Mar_2014_SB= nlgz_Mar_2014.loc[nlgz_Mar_2014['sector']=='SB']

nlgz_Apr_2014_NB= nlgz_Apr_2014.loc[nlgz_Apr_2014['sector']=='NB']
nlgz_Apr_2014_CB= nlgz_Apr_2014.loc[nlgz_Apr_2014['sector']=='CB']
nlgz_Apr_2014_SB= nlgz_Apr_2014.loc[nlgz_Apr_2014['sector']=='SB']

nlgz_Oct_2014_NB= nlgz_Oct_2014.loc[nlgz_Oct_2014['sector']=='NB']
nlgz_Oct_2014_CB= nlgz_Oct_2014.loc[nlgz_Oct_2014['sector']=='CB']
nlgz_Oct_2014_SB= nlgz_Oct_2014.loc[nlgz_Oct_2014['sector']=='SB']

nlgz_Nov_2014_NB= nlgz_Nov_2014.loc[nlgz_Nov_2014['sector']=='NB']
nlgz_Nov_2014_CB= nlgz_Nov_2014.loc[nlgz_Nov_2014['sector']=='CB']
nlgz_Nov_2014_SB= nlgz_Nov_2014.loc[nlgz_Nov_2014['sector']=='SB']

nlgz_Dec_2014_NB= nlgz_Dec_2014.loc[nlgz_Dec_2014['sector']=='NB']
nlgz_Dec_2014_CB= nlgz_Dec_2014.loc[nlgz_Dec_2014['sector']=='CB']
nlgz_Dec_2014_SB= nlgz_Dec_2014.loc[nlgz_Dec_2014['sector']=='SB']

In [117]:
print(str('Wet season median nsmz 2014 (NB):'))
print((nsmz_Jan_2014_NB.median()) + 
      (nsmz_Feb_2014_NB.median()) + 
       (nsmz_Mar_2014_NB.median()) + 
       (nsmz_Apr_2014_NB.median()) + 
       (nsmz_Oct_2014_NB.median()) +
       (nsmz_Nov_2014_NB.median()) +  
       (nsmz_Dec_2014_NB.median()))

print(str('Wet season median nsmz 2014 (CB):'))
print((nsmz_Jan_2014_CB.median()) + 
      (nsmz_Feb_2014_CB.median()) + 
       (nsmz_Mar_2014_CB.median()) + 
       (nsmz_Apr_2014_CB.median()) + 
       (nsmz_Oct_2014_CB.median()) + 
       (nsmz_Nov_2014_CB.median()) + 
       (nsmz_Dec_2014_CB.median()))

print(str('Wet season median nsmz 2014 (SB):'))
print((nsmz_Jan_2014_SB.median()) + 
      (nsmz_Feb_2014_SB.median()) + 
       (nsmz_Mar_2014_SB.median()) + 
       (nsmz_Apr_2014_SB.median()) +
       (nsmz_Oct_2014_SB.median()) +
       (nsmz_Nov_2014_SB.median()) + 
       (nsmz_Dec_2014_SB.median()))

print(str('Wet season median nmdz 2014 (NB):'))
print((nmdz_Jan_2014_NB.median()) + 
      (nmdz_Feb_2014_NB.median()) + 
       (nmdz_Mar_2014_NB.median()) + 
       (nmdz_Apr_2014_NB.median()) +
       (nmdz_Oct_2014_NB.median()) +
       (nmdz_Nov_2014_NB.median()) + 
       (nmdz_Dec_2014_NB.median()))

print(str('Wet season median nmdz 2014 (CB):'))
print((nmdz_Jan_2014_CB.median()) + 
      (nmdz_Feb_2014_CB.median()) + 
       (nmdz_Mar_2014_CB.median()) + 
       (nmdz_Apr_2014_CB.median()) +
       (nmdz_Oct_2014_CB.median()) +
       (nmdz_Nov_2014_CB.median()) + 
       (nmdz_Dec_2014_CB.median()))

print(str('Wet season median nmdz 2014 (SB):'))
print((nmdz_Jan_2014_SB.median()) + 
      (nmdz_Feb_2014_SB.median()) + 
       (nmdz_Mar_2014_SB.median()) + 
       (nmdz_Apr_2014_SB.median()) + 
       (nmdz_Oct_2014_SB.median()) + 
       (nmdz_Nov_2014_SB.median()) + 
       (nmdz_Dec_2014_SB.median()))

print(str('Wet season median nlgz 2014 (NB):'))
print((nlgz_Jan_2014_NB.median()) + 
      (nlgz_Feb_2014_NB.median()) + 
       (nlgz_Mar_2014_NB.median()) + 
       (nlgz_Apr_2014_NB.median()) +
       (nlgz_Oct_2014_NB.median()) +
       (nlgz_Nov_2014_NB.median()) + 
       (nlgz_Dec_2014_NB.median()))

print(str('Wet season median nlgz 2014 (CB):'))
print((nlgz_Jan_2014_CB.median()) + 
      (nlgz_Feb_2014_CB.median()) + 
       (nlgz_Mar_2014_CB.median()) + 
       (nlgz_Apr_2014_CB.median()) +
       (nlgz_Oct_2014_CB.median()) +
       (nlgz_Nov_2014_CB.median()) +
       (nlgz_Dec_2014_CB.median()))

print(str('Wet season median nlgz 2014 (SB):'))
print((nlgz_Jan_2014_SB.median()) + 
      (nlgz_Feb_2014_SB.median()) + 
       (nlgz_Mar_2014_SB.median()) + 
       (nlgz_Apr_2014_SB.median()) +
       (nlgz_Oct_2014_SB.median()) +
       (nlgz_Nov_2014_SB.median()) + 
       (nlgz_Dec_2014_SB.median()))

Wet season median nsmz 2014 (NB):
Unnamed: 0         2.412885e+05
concentration      7.954877e-07
year               1.409800e+04
nsmz_difference    8.551937e-08
dtype: float64
Wet season median nsmz 2014 (CB):
Unnamed: 0         2.589355e+05
concentration      8.371870e-07
year               1.409800e+04
nsmz_difference    6.214095e-08
dtype: float64
Wet season median nsmz 2014 (SB):
Unnamed: 0         2.737615e+05
concentration      8.455842e-07
year               1.409800e+04
nsmz_difference    6.429753e-08
dtype: float64
Wet season median nmdz 2014 (NB):
Unnamed: 0         2.412885e+05
concentration      4.615705e-07
year               1.409800e+04
nmdz_difference    4.280419e-08
dtype: float64
Wet season median nmdz 2014 (CB):
Unnamed: 0         2.589355e+05
concentration      5.620583e-07
year               1.409800e+04
nmdz_difference    6.664598e-08
dtype: float64
Wet season median nmdz 2014 (SB):
Unnamed: 0         2.737615e+05
concentration      7.241276e-07
year             

In [125]:
# Wet season

fig = go.Figure(data=go.Heatmap(
                   z=[[7.95/7.89,8.37/8.67,8.46/8.91], 
                      [4.62/4.65,5.62/6.04,7.24/8.38], 
                      [1/1,1.13/1.15,1.43/1.57]],
                   x=['NB', 'CB', 'SB'],
                   y=['Small zoos', 'Med zoos', 'Large zoos'],
                    colorscale="Viridis",
                    zmin=1, zmax=1.5))                   

fig.update_layout(
    title='Wet (Jan-Apr, Oct-Dec) season proportional differences: 2014 v. 2013')

fig.show()